# Import packages

In [1]:
import os
import cmaps
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from scipy.interpolate import griddata
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from matplotlib.backends.backend_pdf import PdfPages
rcParams['font.family'] = 'sans-serif'
directory   = '/g/data/fu5/eac/20year_run_Junde/jl4424/KC_GS'

# Read mean MKE and EKE from AVISO and BRAN

In [2]:
dataset0   = sio.loadmat(directory+'/Figures/Figure2_mean.mat')
aviso_lon  = dataset0['aviso_lon'][:,:]
aviso_lat  = dataset0['aviso_lat'][:,:]
bran_lon   = dataset0['bran_lon'][:,:]
bran_lat   = dataset0['bran_lat'][:,:]
bran_eke   = dataset0['bran_EKE_mean'][:,:]
aviso_eke  = dataset0['aviso_eke_mean'][:,:]
# Read BRAN mke
dataset1   = xr.open_dataset(directory+'/BRAN/BRAN2020_KC_GS_clim_uvw.nc')
bran_u     = dataset1.clim_u[0,:,:].transpose().values
bran_v     = dataset1.clim_v[0,:,:].transpose().values
bran_mke   = 0.5 * np.sqrt(bran_u * bran_u  + bran_v * bran_v)
bran_u[np.isnan(bran_eke)] = np.nan
bran_v[np.isnan(bran_eke)] = np.nan
bran_mke[np.isnan(bran_eke)] = np.nan
# Read AVISO mke
dataset2   = xr.open_dataset(directory+'/AVISO/AVISO_monthly_SSH.nc')
dataset3   = xr.open_dataset(directory+'/AVISO/AVISO_KC_GS_ugos.nc')
dataset4   = xr.open_dataset(directory+'/AVISO/AVISO_KC_GS_vgos.nc')
aviso_ssh  = dataset2.adt.mean(axis=0).transpose().values
aviso_ugos = dataset3.ugos.mean(axis=0).transpose().values
aviso_vgos = dataset4.vgos.mean(axis=0).transpose().values
aviso_mke  = 0.5 * np.sqrt(aviso_ugos * aviso_ugos + aviso_vgos * aviso_vgos)
#################################################################################################
dataset5 = xr.open_dataset(directory+'/BRAN/BRAN2020_KC_GS_clim_SSH.nc')
bran_ssh = dataset5.clim_ssh.transpose()
# KC_ssh   = bran_ssh[200:502,299:451]
# GS_ssh   = bran_ssh[1600:1902,299:451]

# Plot the spatial distribution of MKE and surface EKE

In [ ]:
##############################################################################################################
labels        = ['a','b','c','d','e','f','g','h']
levels1       = np.linspace(0,0.25,50)
tick_marks1   = np.linspace(0,0.25,6)
cmaps1        = cmocean.cm.matter
fig_ratio     = 0.9
fig           = plt.figure(figsize=(16, 32))
gs            = gridspec.GridSpec(4,2)
labelfont     = 20
padspacescale = 10
labelpadscale = 2
linefont      = 1
scale1        = 1.0
scale2        = 1.0
##############################################################################################################
for i in range(8):
    if i==0:
        lon       = aviso_lon[80:202,120:182]      
        lat       = aviso_lat[80:202,120:182]      
        var_ssh   = aviso_ssh[80:202,120:182]      
        ssh_level = [1.1]
        var_mean  = aviso_mke[80:202,120:182]        
        var_u     = aviso_ugos[80:202,120:182]      
        var_v     = aviso_vgos[80:202,120:182]      
    elif i==1:
        lon       = aviso_lon[640:762,120:182]      
        lat       = aviso_lat[640:762,120:182]  
        var_ssh   = aviso_ssh[640:762,120:182]  
        ssh_level = [0.25]
        var_mean  = aviso_mke[640:762,120:182]      
        var_u     = aviso_ugos[640:762,120:182]       
        var_v     = aviso_vgos[640:762,120:182]   
    elif i==2:
        lon       = bran_lon[200:502,299:451]      
        lat       = bran_lat[200:502,299:451] 
        var_ssh   = bran_ssh[200:502,299:451]
        ssh_level = [0.7]
        var_mean  = bran_mke[200:502,299:451]
        var_u     = bran_u[200:502,299:451]
        var_v     = bran_v[200:502,299:451]
    elif i==3:  
        lon       = bran_lon[1600:1902,299:451]      
        lat       = bran_lat[1600:1902,299:451]      
        var_ssh   = bran_ssh[1600:1902,299:451]
        ssh_level = [0.0]
        var_mean  = bran_mke[1600:1902,299:451]      
        var_u     = bran_u[1600:1902,299:451]      
        var_v     = bran_v[1600:1902,299:451]      
    elif i==4:
        lon       = aviso_lon[80:202,120:182]           
        lat       = aviso_lat[80:202,120:182]       
        var_ssh   = aviso_ssh[80:202,120:182]      
        ssh_level = [1.1] 
        var_mean  = aviso_eke[80:202,120:182] 
    elif i==5:
        lon       = aviso_lon[640:762,120:182]      
        lat       = aviso_lat[640:762,120:182]  
        var_ssh   = aviso_ssh[640:762,120:182]   
        ssh_level = [0.25]
        var_mean  = aviso_eke[640:762,120:182]       
    elif i==6:
        lon       = bran_lon[200:502,299:451]      
        lat       = bran_lat[200:502,299:451]   
        var_ssh   = bran_ssh[200:502,299:451]
        ssh_level = [0.7]
        var_mean  = bran_eke[200:502,299:451]          
    else:
        lon       = bran_lon[1600:1902,299:451]          
        lat       = bran_lat[1600:1902,299:451]      
        var_ssh   = bran_ssh[1600:1902,299:451]
        ssh_level = [0.0]
        var_mean  = bran_eke[1600:1902,299:451]     
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    ax = fig.add_subplot(gs[i])
    l, b, w, h = ax.get_position().bounds
    m   = Basemap(projection='merc',suppress_ticks=True,llcrnrlat=np.nanmin(lat),urcrnrlat=np.nanmax(lat),llcrnrlon=np.nanmin(lon),urcrnrlon=np.nanmax(lon),resolution='c')                
    m.drawparallels(np.arange(15,50, 5),labels=[1,0,0,0],linewidth=linefont,dashes=[2,2],color='.5',fontsize=0.8*labelfont,family='sans-serif')    
    m.drawmeridians(np.arange(0,360, 5),labels=[0,0,0,1],linewidth=linefont,dashes=[2,2],color='.5',fontsize=0.8*labelfont,family='sans-serif')
    x, y = m(lon, lat)
    
    if i<2:
        b   = b
        yy  = np.arange(1, y.shape[0], 4)
        xx  = np.arange(1, x.shape[1], 4)
        CB1 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels1,origin='lower',extend='both')
    elif i<4:
        b   = b+0.06
        yy  = np.arange(1, y.shape[0], 10)
        xx  = np.arange(1, x.shape[1], 10)
        CB2 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels1,origin='lower',extend='both')
    elif i<6:
        b   = b+0.12
        yy  = np.arange(1, y.shape[0], 4)
        xx  = np.arange(1, x.shape[1], 4)
        CB3 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels1,origin='lower',extend='both')
    else:
        b   = b+0.18
        yy  = np.arange(1, y.shape[0], 10)
        xx  = np.arange(1, x.shape[1], 10)
        CB4 = m.contourf(x, y, var_mean,cmap=cmaps1,levels=levels1,origin='lower',extend='both')
        plt.xlabel('Longitude',fontsize=labelfont,labelpad=2*padspacescale,family='sans-serif')
        
    points  = np.meshgrid(yy, xx)
    if np.mod(i,2)==0:  
        cx,cy = m(136.5, 46.0)
        plt.ylabel('Latitude',fontsize=labelfont,labelpad=4.5*padspacescale,family='sans-serif')
        CS  = m.contour(x, y, var_ssh,ssh_level,linewidths=5*linefont,linestyles='solid',colors='tab:gray')
    else:
        cx,cy  = m(276.5, 46.0)
        CS  = m.contour(x, y, var_ssh,ssh_level,linewidths=5*linefont,linestyles='solid',colors='tab:blue')
    
    if i<4:
        if np.mod(i,2)==0:
            Q = m.quiver(x[tuple(points)],y[tuple(points)],var_u[tuple(points)],var_v[tuple(points)],color='xkcd:blue',width=0.003,scale=15.0)
            qk = plt.quiverkey(Q, 0.1, 0.86, 1.0, r'1.0 m s$^{-1}$',labelpos='N',color='xkcd:blue',linewidth=1.2*linefont,fontproperties={'size': 0.5*labelfont})
        else:
            Q = m.quiver(x[tuple(points)],y[tuple(points)],var_u[tuple(points)],var_v[tuple(points)],color='xkcd:blue',width=0.003,scale=15.0)
            qk = plt.quiverkey(Q, 0.1, 0.74, 1.0, r'1.0 m s$^{-1}$',labelpos='N',color='xkcd:blue',linewidth=1.2*linefont,fontproperties={'size': 0.5*labelfont})
  
    m.plot(x[0,:],  y[0,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,0],  y[:,0],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[-1,:], y[-1,:],linewidth=linefont, linestyle='solid', color='k')
    m.plot(x[:,-1], y[:,-1],linewidth=linefont, linestyle='solid', color='k')
    m.drawcoastlines(color='0.1',  linewidth=linefont)
    m.fillcontinents(color='0.9', lake_color='white')
    ax.spines['left'].set_linewidth(labelpadscale)
    ax.spines['top'].set_linewidth(labelpadscale)
    ax.spines['right'].set_linewidth(labelpadscale)
    ax.spines['bottom'].set_linewidth(labelpadscale)
    plt.tick_params(axis='x',which='major',bottom='on',left='on',top='on',right='on',\
                length=25,width=2,colors='black',direction='out')
    plt.text(cx, cy,labels[i],color='xkcd:black', fontsize=labelfont, weight='bold',family='sans-serif')
    if np.mod(i,2)==0:
        ax.set_position([l, b, scale1*w, scale1*h])   
    elif np.mod(i,2)==1:
        ax.set_position([l, b, scale2*w, scale2*h])   
cbaxes1     = fig.add_axes([0.91, 0.74, 0.01, 0.11])
cb1 = plt.colorbar(CB1,orientation='vertical',cax = cbaxes1)
cb1.set_ticks(tick_marks1)
cb1.ax.tick_params(labelsize=0.8*labelfont)
cb1.set_label(r'AVISO MKE (m$^{2}$ s$^{-2}$)', fontsize=0.8*labelfont,labelpad=0,family='sans-serif')
cbaxes2     = fig.add_axes([0.91, 0.60, 0.01, 0.11])
cb2 = plt.colorbar(CB2,orientation='vertical',cax = cbaxes2)
cb2.set_ticks(tick_marks1)
cb2.ax.tick_params(labelsize=0.8*labelfont)
cb2.set_label(r'BRAN MKE (m$^{2}$ s$^{-2}$)', fontsize=0.8*labelfont,labelpad=0,family='sans-serif')
cbaxes3     = fig.add_axes([0.91, 0.46, 0.01, 0.11])
cb3 = plt.colorbar(CB3,orientation='vertical',cax = cbaxes3)
cb3.set_ticks(tick_marks1)
cb3.ax.tick_params(labelsize=0.8*labelfont)
cb3.set_label(r'AVISO $\overline{EKE}$ (m$^{2}$ s$^{-2}$)', fontsize=0.8*labelfont,labelpad=0,family='sans-serif')
cbaxes4     = fig.add_axes([0.91, 0.32, 0.01, 0.11])
cb4 = plt.colorbar(CB4,orientation='vertical',cax = cbaxes4)
cb4.set_ticks(tick_marks1)
cb4.ax.tick_params(labelsize=0.8*labelfont)
cb4.set_label(r'BRAN $\overline{EKE}$ (m$^{2}$ s$^{-2}$)', fontsize=0.8*labelfont,labelpad=0,family='sans-serif')
fig.savefig(directory+'/Figures/FigureS1_mke_eke.png',dpi=300,bbox_inches = 'tight')